In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# Development of a Jukes-Cantor pipeline for generation of synthetic data

## Graph functions

## Jukes-Cantor model functions

## Testing 